In [ ]:
import jwt  # Install with: pip install pyjwt
import time
import requests
import pandas as pd

TEAM_ID = '6A3CSCZ9M6'
KEY_ID = 'AVQK9UTY3R'
PRIVATE_KEY_PATH = r'E:\VK-Entertainment\AuthKey_AVQK9UTY3R.p8'

def generate_apple_music_token():
    with open(PRIVATE_KEY_PATH, "r") as key_file:
        private_key = key_file.read()
    
    payload = {
        "iss": TEAM_ID,
        "iat": int(time.time()),
        "exp": int(time.time()) + 3600,  
    }

    token = jwt.encode(payload, private_key, algorithm="ES256", headers={"alg": "ES256", "kid": KEY_ID})
    return token

APPLE_MUSIC_TOKEN = generate_apple_music_token()



In [9]:
df_apple_music

,name,Chức danh,link,Thể loại nhạc,Followers,uri
0,Bình Gold,artist,https://music.apple.com/us/artist/b%C3%ACnh-go...,Hip-Hop/Rap,None,1486157747


In [ ]:
def get_artist_albums(artist_id, storefront="us"):
    """ Lấy danh sách album của nghệ sĩ """
    url = f"https://api.music.apple.com/v1/catalog/{storefront}/artists/{artist_id}/albums"
    headers = {"Authorization": f"Bearer {APPLE_MUSIC_TOKEN}"}

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        albums = response.json().get("data", [])
        return [
            {
                "album_id": album["id"],
                "album_name": album["attributes"]["name"],
                "release_date": album["attributes"]["releaseDate"],
                "medium": "single" if album["attributes"]["isSingle"] else "album",
                "genre": ", ".join(album["attributes"].get("genreNames", [])),
                "album_url": album["attributes"]["url"]
            }
            for album in albums
        ]
    return []

def get_album_tracks(album_id, storefront="us"):
    """ Lấy danh sách bài hát trong album """
    url = f"https://api.music.apple.com/v1/catalog/{storefront}/albums/{album_id}/tracks"
    headers = {"Authorization": f"Bearer {APPLE_MUSIC_TOKEN}"}

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        tracks = response.json().get("data", [])
        return [
            {
                "album_id": album_id,
                "tracklist": track["attributes"]["name"],
                "track_url": track["attributes"]["url"],
                "featured_artists": track["attributes"].get("artistName", "None")  
            }
            for track in tracks
        ]
    return []


# 🔹 Lấy thông tin album của nghệ sĩ
albums_data = []
for artist_id in artist_ids:
    albums_data.extend(get_artist_albums(artist_id))

df_albums = pd.DataFrame(albums_data)

# 🔹 Lấy thông tin bài hát của album
tracks_data = []
for album in albums_data:
    album_id = album["album_id"]
    for track in get_album_tracks(album_id):
        track.update({
            "album_name": album["album_name"],
            "release_date": album["release_date"],
            "status_code": "Normal",
            "class": "digital",
            "genre": album["genre"],
            "medium": album["medium"]
        })
        tracks_data.append(track)

df_tracks = pd.DataFrame(tracks_data)


In [12]:
df_tracks

,album_id,tracklist,track_url,featured_artists,album_name,release_date,status_code,class,genre,medium,popularity
0,1606483780,"Em iu (feat. Wxrdie, Bình Gold & 2pillz)",https://music.apple.com/us/album/em-iu-feat-wx...,Andree Right Hand,"Em iu (feat. Wxrdie, Bình Gold & 2pillz) - Single",2022-01-30,Normal,digital,"Hip-Hop/Rap, Music",single,None
1,1771437409,Đổi Tư Thế,https://music.apple.com/us/album/%C4%91%E1%BB%...,Bình Gold & Andree Right Hand,Đổi Tư Thế - Single,2024-10-02,Normal,digital,"Hip-Hop/Rap, Music",single,None
2,1748724965,OBGTLH,https://music.apple.com/us/album/obgtlh/174872...,Bình Gold & Lil Shady,OBGTLH - Single,2020-08-06,Normal,digital,"Hip-Hop/Rap, Music",single,None
3,1748785808,Trăng Hoa Mây Mưa,https://music.apple.com/us/album/tr%C4%83ng-ho...,Bình Gold,Trăng Hoa Mây Mưa - Single,2024-01-13,Normal,digital,"Hip-Hop/Rap, Music",single,None
4,1749301687,Tuổi Gì Mà Chẳng Thích Lì Xì (feat. Bình Gold),https://music.apple.com/us/album/tu%E1%BB%95i-...,Bích Phương,Tuổi Gì Mà Chẳng Thích Lì Xì (feat. Bình Gold)...,2020-01-16,Normal,digital,"Pop, Music",single,None
5,1748720681,Lái Máy Bay,https://music.apple.com/us/album/l%C3%A1i-m%C3...,Bình Gold,Lái Máy Bay - Single,2019-07-01,Normal,digital,"Hip-Hop/Rap, Music",single,None
6,1748728100,BCDBL,https://music.apple.com/us/album/bcdbl/1748728...,Bình Gold,BCDBL - Single,2022-02-02,Normal,digital,"Hip-Hop/Rap, Music",single,None
7,1787527746,Đổi Tư Thế (KZuy Remix),https://music.apple.com/us/album/%C4%91%E1%BB%...,Bình Gold & Andree Right Hand,Đổi Tư Thế (KZuy Remix) - Single,2024-12-27,Normal,digital,"Pop, Music",single,None
8,1748728406,Trơn,https://music.apple.com/us/album/tr%C6%A1n/174...,Bình Gold,Trơn - Single,2021-02-02,Normal,digital,"Hip-Hop/Rap, Music, Hip-Hop",single,None
9,1756496414,Em iu,https://music.apple.com/us/album/em-iu/1756496...,"Ricky Star, Andree Right Hand, Wxrdie, Bình Go...",Em iu - Single,2022-08-29,Normal,digital,"Hip-Hop/Rap, Music",single,None


In [13]:
df_tracks.to_excel("binh_gold_apple.xlsx")